In [17]:
%load_ext autoreload
%autoreload 2

In [8]:
import anndata

adata = anndata.read_h5ad("../../data/raw/bowel_disease.h5ad")

In [26]:
import scanpy as sc

sc.pp.calculate_qc_metrics(adata, inplace=True)
adata.obs["n_counts"] = adata.obs.total_counts

In [23]:
from mmcontext.pp.utils import add_ensembl_ids

add_ensembl_ids(adata)

In [28]:
adata.write_h5ad("../../data/raw/bowel_disease.h5ad")

In [31]:
from geneformer import TranscriptomeTokenizer

tk = TranscriptomeTokenizer({"cluster_label": "cluster_label"}, nproc=16)
tk.tokenize_data("../../data/raw/", "../../data/geneformer_datasets", "bowel_disease", file_format="h5ad")

Tokenizing ../../data/raw/bowel_disease.h5ad


/Users/mengerj/repos/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/Users/mengerj/repos/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


../../data/raw/bowel_disease.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [33]:
from datasets import load_from_disk

dataset = load_from_disk("../../data/geneformer_datasets/bowel_disease.dataset")
print(dataset)

Dataset({
    features: ['input_ids', 'cluster_label', 'length'],
    num_rows: 11175
})


In [36]:
dataset["cluster_label"]

['Enterocytes with TMIGD1, MEP1A, GSTA1 Expression',
 'Activated CD8+ T Cells in Intestine',
 'Activated CD8+ T Cells in Intestine',
 'Proliferating Intestinal Transit Amplifying Cells',
 'Active Mucus-Secreting Goblet Cells in Ileum',
 'Proliferating Colonic Transit Amplifying Cells',
 'Active Intestinal Goblet Cells in Mucosal Defense',
 'Active Mucus-Secreting Goblet Cells in Ileum',
 'Cycling ileal epithelial precursor cells',
 'Proliferating Colonic Transit Amplifying Cells',
 'Intestinal Tuft Cells: Function and Immunity',
 'Activated CD8+ T Cells in Intestine',
 'Enterocytes with TMIGD1, MEP1A, GSTA1 Expression',
 'Activated CD8+ T Cells in Intestine',
 'Proliferating Colonic Transit Amplifying Cells',
 'Proliferating Colonic Transit Amplifying Cells',
 'Cycling ileal epithelial precursor cells',
 'Activated CD8+ T Cells in Intestine',
 'Proliferating Intestinal Transit Amplifying Cells',
 'Healthy Large Intestine Epithelial Cells',
 'Active Mucus-Secreting Goblet Cells in Ileum

In [40]:
from geneformer import EmbExtractor

# zero shot retrieval of embeddings
embex = EmbExtractor(
    model_type="CellClassifier",
    num_classes=3,
    max_ncells=1000,
    emb_layer=0,
    emb_label=["disease", "cell_type"],
    labels_to_plot=["cell_type"],
    forward_batch_size=200,
    nproc=16,
    token_dictionary_file="../../../Geneformer/geneformer/gene_dictionaries_30m/token_dictionary_gc30M.pkl",
)  # change from current default dictionary for 30M model series

# extracts embedding from input data
# input data is tokenized rank value encodings generated by Geneformer tokenizer (see tokenizing_scRNAseq_data.ipynb)
# example dataset for 30M model series: https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/tree/main/example_input_files/cell_classification/disease_classification/human_dcm_hcm_nf.dataset
embs = embex.extract_embs(
    "../../../Geneformer/fine_tuned_models/gf-6L-30M-i2048_CellClassifier_cardiomyopathies_220224",  # example 30M fine-tuned model
    "../../data/geneformer_datasets/bowel_disease.dataset",
    "../../ouputs/bowel_disease_embeddings",
    "gf-6L-30M-i2048_CellClassifier_cardiomyopathies_220224",
)

FileNotFoundError: Directory ../../data/raw/bowel_disease.h5ad is neither a `Dataset` directory nor a `DatasetDict` directory.